# Introduction to deterministic and stochastic modelling of simple biological systems: part 1
## Dr Marc Sturrock

DiffEqBiological.jl is a domain specific language (DSL) for writing chemical
reaction networks in Julia. The generated chemical reaction network model can
then be translated into a variety of mathematical models which can be solved
using components of the broader
[DifferentialEquations.jl](http://juliadiffeq.org/) ecosystem. Biomodelling.jl is a package for simulating gene expression networks, cell growth, division and partitioning. 

In this tutorial we'll provide an introduction to using DiffEqBiological to
specify chemical reaction networks, and then to solve deterministic and stochastic models generated from them. In the second part, we will then use Biomodelling.jl to understand the concepts of intrinsic and extrinsic noise.


Let's start by using the DiffEqBiological
`@reaction_network` macro to specify a simply chemical reaction network; the
reactions associated with the central dogma of molecular biology.

We first add and import the basic packages we'll need, and use Plots.jl for making
figures:

In [ ]:
#First load packages
using DifferentialEquations, DiffEqBiological, Plots, Latexify
gr()

We now construct the reaction network. The basic types of arrows and predefined rate laws one can use are discussed in detail within the DiffEqBiological [Chemical Reaction Models documentation](http://docs.juliadiffeq.org/latest/models/biological.html). Here we use a mix of zero order and first order rate laws. Note, $\varnothing$ corresponds to the empty state, and is used for zeroth order production and first order degradation reactions:

In [ ]:
geneexpression = @reaction_network begin
    b_1, 0 --> mRNA
    d_1, mRNA --> 0
    b_2, mRNA --> mRNA + protein
    d_2, protein --> 0
end b_1 d_1 b_2 d_2

We can use Latexify to look at the corresponding reactions and understand the generated rate laws for each reaction


In [ ]:
latexify(geneexpression; env=:chemical)

We can also use Latexify to look at the corresponding ODE model for the chemical system which is created using the law of mass action

In [ ]:
latexify(geneexpression)

To solve the ODEs we need to specify the values of the parameters in the model, the initial condition, and the time interval to solve the model on. To do this it helps to know the orderings of the parameters and the species. Parameters are ordered in the same order they appear after the end statement in the `@reaction_network` macro. Species are ordered in the order they first appear within the @reaction_network macro. We can see these orderings using the speciesmap and paramsmap functions:

In [ ]:
speciesmap(geneexpression)

In [ ]:
paramsmap(geneexpression)

## Solving a deterministic version of the system: ODEs 

Knowing these orderings, we can create parameter and initial condition vectors, and setup the ODEProblem we want to solve:

In [ ]:
# parameters [b_1,d_1,b_2,d_2]
p = (10.0,0.1,1.0,0.001)

# initial condition [mRNA0,protein0]
u₀ = [0.0,0.0]

# time interval to solve on
tspan = (0., 5000.0)

# create the ODEProblem we want to solve
prob = ODEProblem(geneexpression, u₀, tspan, p)

At this point we are all set to solve the ODEs. We can now use any ODE solver from within the DiffEq package. We'll just use the default DifferentialEquations solver for now, and then plot the solutions:

In [ ]:
sol1 = solve(prob, saveat=10.)
plot(sol1,layout=(1,2),legend=:bottomright,ylabel=["mRNA" "protein"])

For more on choices of ODE solvers, see the JuliaDiffEq [documentation](http://docs.juliadiffeq.org/latest/solvers/ode_solve.html)

## Solving a stochastic version of the system: Stochastic simulation algorithm


Let's now look at a stochastic chemical kinetics model of the repressilator, modeling it with jump processes. Here we will construct a DiffEqJump JumpProblem that uses Gillespie's Direct method, and then solve it to generate one realization of the jump process:

In [ ]:
# first we redefine the initial condition to be integer valued
u₀ = [10,1000]

# next we create a discrete problem to encode that our species are integer valued:
dprob = DiscreteProblem(geneexpression, u₀, tspan, p)

# now we create a JumpProblem, and specify Gillespie's Direct Method as the solver:
jprob = JumpProblem(dprob, Direct(), geneexpression, save_positions=(false,false))

# now let's solve and plot the jump process:
sol2 = solve(jprob, SSAStepper(), saveat=10.)
plot(sol2,layout=(1,2),legend=:bottomright,ylabel=["mRNA" "protein"])

While SSAs generate exact realisations for stochastic chemical kinetics jump process models, a problem with the SSA is that it can be prohibitively slow, particularly when large numbers of molecules are simulated.  $\tau$-leaping methods offer a fast alternative by discretising in time the underlying time-change representation of the stochastic process. The DiffEqJump package has limited support for $\tau$-leaping methods in the form of the basic Euler's method type approximation proposed by Gillespie (Biomodelling.jl contains more tau leaping algorithms). We can simulate a $\tau$-leap approximation to the repressilator by using the  RegularJump representation of the network to construct a JumpProblem:

In [ ]:
rjs = regularjumps(geneexpression)
lprob = JumpProblem(dprob, Direct(), rjs)
lsol = solve(lprob, SimpleTauLeaping(), dt=.1)
plot(lsol,layout=(1,2),legend=:bottomright,ylabel=["mRNA" "protein"])

By taking the mean of the stochastic solutions across 100 different simulations (representing gene expression in 100 different cells), we can see that the ordinary differential equation model gives an accurate mean-field representation of gene expression for a cell population. This is not parameter set dependent and you can try changing p defined above to whatever you like and rerunning cells to confirm this.

In [ ]:
output = zeros(2,50001,10)
for i = 1:10
    lsol = solve(lprob, SimpleTauLeaping(), dt=.1)
    output[:,:,i] = lsol[1:2,:]
end
using Statistics
mean_stochastic_sol = mean(output,dims=3)[:,:,1]
plot(sol1)
plot!(lsol.t,mean_stochastic_sol',linestyle=:dash,linewidth=4,labels=["mRNA stochastic" "protein stochastic"])

Can we say this in general? Is the ODE model solution always consistent with the mean of the stochastic model?

## mRNA-miRNA-protein system

The paper of [Paulsson et al.](https://www.pnas.org/content/97/13/7148) provides a neat example of how even a simple system can yield different results for the stochatic and deterministic analogues. 

First we define the system, which consists of zero and first order reactions as well as a single second order reaciton. This system can be interpreted in a number of different ways, for example, $I$ can be interpreted as an mRNA molecule, $P$ as a protein molecule, and $S$ as a microRNA molecule  

In [ ]:
ps = @reaction_network begin
   k, 0 --> I
   ka*S, I --> 0
   kp, I --> P
   1, P --> 0
   b, 0 --> S
   d, S --> 0
end k ka kp b d

The system can be expressed as stochastic chemical reactions in the following way

In [ ]:
latexify(ps; env=:chemical)

Or as deterministic ODEs as the following

In [ ]:
latexify(ps)

First we can solve the deterministic ODE system

In [ ]:
# parameters [k,ka,kp,b,d]
params = (1000.0,100.0,100.0,1.0,2.0)

#initial conditions
u0 = [1.0, 1.0, 1.0]

# time interval to solve on
tspan = (0., 100.)

# create the ODEProblem we want to solve
oprob = ODEProblem(ps, u0, tspan, params)

# time points to output solution at
timestep = 1.0
ts = collect(0:timestep:maximum(tspan));

Solving the ODE system leads to

In [ ]:
ode_sol  = solve(oprob, alg_hint=:stiff,saveat=ts);

Solving the corresponding stochastic system for 10 different cells, averaging and plotting alongside the ODE results leads to

In [ ]:
dprob = DiscreteProblem(ps, u0, tspan, params)
jprob = JumpProblem(dprob, Direct(), ps, save_positions = (false, false))
output = zeros(Int64,100, length(ts));
for i = 1:100
   jsol = solve(jprob, SSAStepper(), saveat = ts)
   output[i,:] = jsol[2,:]
end

plot(ode_sol.t,output',title="",labels="",xlabel="",ylabel="number of protein molecules",grid=false,linewidth=2.0)
plot!(ode_sol.t,mean(output,dims=1)',label="mean SSA",linewidth=6.0,c=:blue)
plot!(ode_sol.t,ode_sol[2,:],label="ODE",linestyle=:dash,linewidth=6.0,c=:black,xlabel="time",title="b = $(params[4]), d = $(params[5])",legend=:bottomright)

We see there is a discrepancy between the mean number of protein molecules between the stochastic system and solution of the deterministic system. However, we find that by increasing the reaction rates (while keeping the number of miRNA molecules constant), we get good agreement. Hence for some stochastic systems 

In [ ]:
params = (1000.0,100.0,100.0,1000.0,2000.0)
oprob = ODEProblem(ps, u0, tspan, params)
dprob = DiscreteProblem(ps, u0, tspan, params)
jprob = JumpProblem(dprob, Direct(), ps, save_positions = (false, false))
output = zeros(Int64,100, length(ts));
for i = 1:100
   jsol = solve(jprob, SSAStepper(), saveat = ts)
   output[i,:] = jsol[2,:]
end

plot(ode_sol.t,output',title="",labels="",xlabel="",ylabel="number of protein molecules",grid=false,linewidth=2.0)
plot!(ode_sol.t,mean(output,dims=1)',label="mean SSA",linewidth=6.0,c=:blue)
plot!(ode_sol.t,ode_sol[2,:],label="ODE",linestyle=:dash,linewidth=6.0,c=:black,xlabel="time",title="b = $(params[4]), d = $(params[5])",legend=:bottomright,ylims=(0,1000))

As soon as a stochastic system contains a second order reaction (e.g. A + B --> C) it is possible that the mean of the stochastic system will deviate from the corresponding ODE system.  

## Repressilator

The repressilator is a genetic regulatory network consisting of at least one feedback loop with at least three genes, each expressing a protein that represses the next gene in the loop. In biological research, repressilators have been used to build cellular models and understand cell function. There are both artificial and naturally-occurring repressilators. Recently, the naturally-occurring repressilator clock gene circuit in Arabidopsis thaliana (A. thaliana) and mammalian systems have been studied. The repressilator can be defined as

In [ ]:
repressilator = @reaction_network begin
    hillr(P₃,α,K,n), ∅ --> m₁
    hillr(P₁,α,K,n), ∅ --> m₂
    hillr(P₂,α,K,n), ∅ --> m₃
    (δ,γ), m₁ ↔ ∅
    (δ,γ), m₂ ↔ ∅
    (δ,γ), m₃ ↔ ∅
    β, m₁ --> m₁ + P₁
    β, m₂ --> m₂ + P₂
    β, m₃ --> m₃ + P₃
    μ, P₁ --> ∅
    μ, P₂ --> ∅
    μ, P₃ --> ∅
end α K n δ γ β μ;

Again we can use Latexify to look at the corresponding reactions and understand the generated rate laws for each reaction

In [ ]:
latexify(repressilator; env=:chemical)

We can also use Latexify to look at the corresponding ODE model for the chemical system

In [ ]:
latexify(repressilator)

First we can solve the deterministic ODE system

In [ ]:
# parameters [α,K,n,δ,γ,β,μ]
p = (.5, 40, 2, log(2)/120, 5e-3, 20*log(2)/120, log(2)/60)

# initial condition [m₁,m₂,m₃,P₁,P₂,P₃]
u₀ = [0.,0.,0.,20.,0.,0.]

# time interval to solve on
tspan = (0., 10000.)
timestep = 1.0
ts = collect(0:timestep:maximum(tspan));

# create the ODEProblem we want to solve
oprob = ODEProblem(repressilator, u₀, tspan, p)

In [ ]:
sol = solve(oprob, saveat=ts)
plot(sol)

The solution looks oscillatory in nature, we can see if the same is true for the mean of trajectories from the stochastic model...

In [ ]:
# solve JumpProblem using Gillespie’s Direct Method
u0 =  [0,0,0,20,0,0] # change to int type for stochastic system..
maxcell = 1000 #number of cells to simulate for
dprob = DiscreteProblem(repressilator, u0, tspan, p)
jprob = JumpProblem(dprob, Direct(), repressilator, save_positions = (false, false))
output = zeros(Int64,maxcell, length(ts),6);
for i = 1:maxcell
   jsol = solve(jprob, SSAStepper(), saveat = ts)
    for j = 1:6
       output[i,:,j] = jsol[j,:]
    end
end

gr(size=(1200,600))
plot(sol.t,mean(output,dims=1)[1,:,:],title="",label=["" "" "" "" "" "mean SSA"],xlabel="",ylabel=["number of molecules" "" "" "number of molecules" "" ""],grid=false,linewidth=6.0,layout=(2,3))
plot!(sol.t,output[1,:,:],title="",label=["" "" "" "" "" "single SSA"],grid=false,linewidth=2.0,layout=(2,3))
plot!(sol,label=["" "" "" "" "" "ODE"],linestyle=:dash,linewidth=6.0,c=:black,legend=:topright,xlabel=["" "" "" "time" "time" "time"],title=["m1" "m2" "m3" "P1" "P2" "P3"],layout=(2,3),grid=false)

Here we see qualitative (but not quantitative) agreement between the deterministic ODE solution and a single trajectory of the stochastic system. However, the mean of several stochastic simulations differs both qualitatively and quantitatively from the ODE solutions. 